# Diagram zonder interactie

Dit is een notebook om een non-interactieve grafiek te tonen van de materiaalstromen grondstoffen zonnepanelen. De huidige vorm van de grafiek is ter illustratie van een concept, de data is deels verzonnen. 

De grafiek is gebasseerd op [de R-ladder, zie deze plaat van het CBS onder 2.1.1](https://www.cbs.nl/nl-nl/longread/de-nederlandse-economie/2020/circulaire-economie-in-nederland/2-circulaire-economie-en-materiaalstromen). 

De meeste data komt uit [de rapportage 2021 van het WEEE register](https://www.nationaalweeeregister.nl/assets/uploads/PDF/2022/Rapportage%202021_V20220627.pdf). 

Als een gebruiker de muis op een stroom of knoop houdt, verschijnt er een zogenaamde hoverlabel, waar de hoeveelheid materiaal in tonnen per jaar wordt getoond en de bron van de data. 



In [2]:
import plotly.graph_objects as go

nodes = [("Grondstof",'rgba(127, 127, 127, 0.4)'), #0
         ("Vooraad Markt",'rgba(188, 189, 34, 0.8)'), #1
         ("In gebruik",'rgba(44, 160, 44, 0.8)'), #2
         ("Buiten gebruik",'rgba(214, 39, 40, 0.8)'), #3
         ("Hergebruik",'rgba(140, 86, 75, 0.8)'), #4
         ("Refurbish<br>Repair",'rgba(31, 119, 180, 0.8)'), #5
         ("Recycle",'rgba(255, 127, 14, 0.8)'), #6
         ("Manufacture",'rgba(148, 103, 189, 0.8)'), #7
         ("Grondstof",'rgba(127, 127, 127, 0.4)'), #8
         ("Waste",'rgba(27, 27, 27, 0.8)'), #9
         ("Export",'rgba(150, 150, 150, 0.4)')] #10

links = [
            ( 0,  1,  243707 ), #https://www.nationaalweeeregister.nl/assets/uploads/PDF/2022/Rapportage%202021_V20220627.pdf
            ( 1,  2,  (243707 - 96698) ), #WEEE
            ( 2,  3,  404+200+493+330 ),
            ( 3,  4,  404 ), #WEEE
            ( 3,  5,  200 ), # zelf verzonnen
            ( 3,  6,  493+330 ), #WEEE
            ( 4,  7,  404 ), #WEEE
            ( 5,  7,  200 ), # zekf verzonnenDit 
            ( 6,  8,  330 ), #WEEE
            ( 6,  9,  493 ), #WEEE
            ( 7,  2,  404+200 ), #WEEE + zelf verzonnen 200 3->5
            ( 1,  10,  96698)]  #https://www.nationaalweeeregister.nl/assets/uploads/PDF/2022/Rapportage%202021_V20220627.pdf

def decrease_alpha(color):
    current_alpha = color.split(', ')[-1]
    current_alpha = float(current_alpha[:-1])
    new_alpha = current_alpha/2
    new_color = color.split(', ')[:-1]
    new_color.append('{:.1f})'.format(new_alpha))
    return ', '.join(new_color)

links = [(l[0],l[1],l[2],decrease_alpha(nodes[l[1]][1])) for l in links]

values = {0:243707}

for node in range(1,len(nodes)):
    values[node] = sum([l[2] for l in links if l[1]==node])
values[2] = 147009



fig = go.Figure(go.Sankey(
    valueformat = ".0f",
    valuesuffix = " ton",
    arrangement = "snap",
    orientation='h',
    node = {
        "label": [node[0] for node in nodes],
        "color": [node[1] for node in nodes],
        'pad':10,
        'customdata':[values[i] for i in range(len(nodes))],
        'hovertemplate':'De vooraad van %{label} is %{customdata} ton<br />Bron: <a href="https://plot.ly/">WEEE rapport</a><extra></extra>'},  # 10 Pixels
    link = {
        "source": [l[0] for l in links],
        "target": [l[1] for l in links],
        "value":  [l[2]/40 if l[0] in [0,1] else l[2] for l in links],
        'customdata':[l[2] for l in links],
        "color":  [l[3] for l in links],
        'hovertemplate':'Stroom van %{source.label}<br />'+
        'naar %{target.label}<br />is %{customdata} ton<br />Bron: <a href="https://plot.ly/">WEEE rapport</a><extra></extra>'}))
fig.update_layout(title_text="R-ladder Grondstoffen zonnepanelen ton/pj 2021", font_size=10)
fig.write_html("r_ladder_zonnepanelen_ton_pj_2021.html")
fig.show()